## Восстановление матриц

Данная работа выполнена в рамках курса по методам оптимизации в Московском Физико-Техническом Институте. Работа состоит из двух частей: теоретического отсчета и практической части, в этом блокноте будет выполнена практическая часть. 

In [1]:
import numpy as np
import scipy.stats as sps
from sklearn.utils.extmath import randomized_svd, svd_flip
from scipy.sparse.linalg import svds

Сначала имплементируем алгоритм Soft-Impute, описание которого можно встретить в книге [Statistical Learning with Sparsity](https://hastie.su.domains/StatLearnSparsity_files/SLS_corrected_1.4.16.pdf) Напомню, что решение задачи восстановления матрицы мы свели к 
$$\min_{M} \left\{ \frac{1}{2} \sum_\limits{(i, j) \in \Omega}
{(z_{ij} - m_{ij})^2 + \lambda ||M||_*} \right\}$$
Эта задача называется спектральной регуляризаций.

In [2]:
def projection(omega, X):
    return (omega * X)

def anti_projection(omega, X):
    return  ((1 - omega) * X)

In [3]:
def s_lambda_operator(X, lambda_):
    U, D, VH = np.linalg.svd(X, full_matrices=False)
    D -= lambda_
    D[D < 0] = 0 
    return np.dot(U * D, VH)

In [4]:
def naive_algo(Z, omega, lambda_=7.5, eps=1/3, theta=0.4, max_iter=1000):
    iter_ = max_iter
    answers = []
    Z_old = np.zeros_like(Z)
    while iter_ and ((len(answers) < 2) or \
                     (np.linalg.norm(answers[-1] - answers[-2]) /\
                                 np.linalg.norm(answers[-1]) > eps)):
        iter_ -= 1
        Z_old = s_lambda_operator(projection(omega, Z) \
                                  + anti_projection(omega, Z_old), lambda_)
        answers.append(Z_old)
        lambda_ /= theta
    return answers[-1]

Данный алгоритм теоретически сходится, однако стоит заметить, что при подсчете матриц с большой нормой мы будем проводить вычисления с маленькой точностью или попросту переполнимся, поэтому всё сломается.

Проверим корректность данного алгоритма на нескольких примерах

In [5]:
Z = np.arange(100).reshape(10,10)
omega = np.ones(100).reshape(10,10)
omega[2, 3] = 0 #маска, явно указывает, какие числа мы как будто бы стерли
omega[3, 3] = 0
omega[4, 3] = 0
omega[3, 5] = 0
omega[4, 5] = 0
print(Z)
print(projection(omega, Z))

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]
 [60 61 62 63 64 65 66 67 68 69]
 [70 71 72 73 74 75 76 77 78 79]
 [80 81 82 83 84 85 86 87 88 89]
 [90 91 92 93 94 95 96 97 98 99]]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [20. 21. 22.  0. 24. 25. 26. 27. 28. 29.]
 [30. 31. 32.  0. 34.  0. 36. 37. 38. 39.]
 [40. 41. 42.  0. 44.  0. 46. 47. 48. 49.]
 [50. 51. 52. 53. 54. 55. 56. 57. 58. 59.]
 [60. 61. 62. 63. 64. 65. 66. 67. 68. 69.]
 [70. 71. 72. 73. 74. 75. 76. 77. 78. 79.]
 [80. 81. 82. 83. 84. 85. 86. 87. 88. 89.]
 [90. 91. 92. 93. 94. 95. 96. 97. 98. 99.]]


In [6]:
result = naive_algo(Z, omega, lambda_=15, eps=0.2, theta=0.9)
print(np.round(anti_projection(omega, result) + projection(omega, Z),
               2))

[[ 0.    1.    2.    3.    4.    5.    6.    7.    8.    9.  ]
 [10.   11.   12.   13.   14.   15.   16.   17.   18.   19.  ]
 [20.   21.   22.   17.21 24.   25.   26.   27.   28.   29.  ]
 [30.   31.   32.    8.87 34.   11.74 36.   37.   38.   39.  ]
 [40.   41.   42.   11.65 44.   15.32 46.   47.   48.   49.  ]
 [50.   51.   52.   53.   54.   55.   56.   57.   58.   59.  ]
 [60.   61.   62.   63.   64.   65.   66.   67.   68.   69.  ]
 [70.   71.   72.   73.   74.   75.   76.   77.   78.   79.  ]
 [80.   81.   82.   83.   84.   85.   86.   87.   88.   89.  ]
 [90.   91.   92.   93.   94.   95.   96.   97.   98.   99.  ]]


Видно, что такая реализация алгоритма весьма сомнительна, по крайней мере на данном примере. Рассмотрим еще варианты.

In [7]:
Z = np.arange(100).reshape(10,10)
Z = Z % 20
omega = np.ones(100).reshape(10,10)
omega[2, 3] = 0 #маска, явно указывает, какие числа мы как будто бы стерли
omega[3, 3] = 0
omega[4, 3] = 0
omega[3, 5] = 0
omega[4, 5] = 0
print(Z)
print(projection(omega, Z))

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  0.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12.  0. 14.  0. 16. 17. 18. 19.]
 [ 0.  1.  2.  0.  4.  0.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]]


In [8]:
result = naive_algo(Z, omega, lambda_=6, eps=0.5, theta=0.4)
print(np.round(anti_projection(omega, result) + projection(omega, Z),
               2))

[[ 0.    1.    2.    3.    4.    5.    6.    7.    8.    9.  ]
 [10.   11.   12.   13.   14.   15.   16.   17.   18.   19.  ]
 [ 0.    1.    2.    3.17  4.    5.    6.    7.    8.    9.  ]
 [10.   11.   12.    8.22 14.    9.91 16.   17.   18.   19.  ]
 [ 0.    1.    2.    2.89  4.    3.48  6.    7.    8.    9.  ]
 [10.   11.   12.   13.   14.   15.   16.   17.   18.   19.  ]
 [ 0.    1.    2.    3.    4.    5.    6.    7.    8.    9.  ]
 [10.   11.   12.   13.   14.   15.   16.   17.   18.   19.  ]
 [ 0.    1.    2.    3.    4.    5.    6.    7.    8.    9.  ]
 [10.   11.   12.   13.   14.   15.   16.   17.   18.   19.  ]]


При меньших числах видно, что результат оказался немного более убедительным. Посмотрим на относительное отклонение.

In [9]:
fix_zero = 1e-06
print((np.round(anti_projection(omega, result) + projection(omega, Z),
               2) + fix_zero)/ (Z + fix_zero))

[[1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.05666665 1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         0.63230772 1.         0.66066669
  1.         1.         1.         1.        ]
 [1.         1.         1.         0.96333335 1.         0.69600006
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.      

Хорошо видно, что малые значения мы приближаем очень хорошо, тогда как более крупные гораздо хуже.

### Более эфективный алгоритм

Перейдем теперь к реализации Soft-thresholding algorithm, описанной в статье [Cai, Candes, Shen 2008](https://arxiv.org/pdf/0810.3286.pdf)

In [10]:
def svd_first_k(M, k):
    U, S, V = svds(M, k=min(k, min(M.shape)-1))
    U, V = svd_flip(U[:, ::-1], V[::-1])
    return U, S[::-1], V

def svt_algo(Z, omega, tau=None, delta=None, eps=1e-03, max_iter=1000):
    if not tau:
        tau = 5 * np.sum(Z.shape) / 2
    if not delta:
        delta = 1.2 * np.prod(Z.shape) / np.sum(omega)

    Z_old = np.zeros_like(Z)
    Y = delta * omega * (Z - Z_old)
    
    r_prev = 0
    
    for k in range(max_iter):
        sk = r_prev + 1
        (U, S, V) = svd_first_k(Y, sk,)
        while np.min(S) >= tau:
            sk = sk + 1
            (U, S, V) = svd_first_k(Y, sk)
        shrink_S = np.maximum(S - tau, 0)
        r_prev = np.count_nonzero(shrink_S)
        Z_old = np.linalg.multi_dot([U, np.diag(shrink_S), V])
        Y += delta * omega * (Z - Z_old)
        
        if np.linalg.norm(omega * (Z - Z_old )) \
                    / np.linalg.norm(omega * Z) < eps:
            break

    return Z_old

In [11]:
Z = np.arange(100).reshape(10,10)
omega = np.ones(100).reshape(10,10)
omega[2, 3] = 0 #маска, явно указывает, какие числа мы как будто бы стерли
omega[3, 3] = 0
omega[4, 6] = 0

Z = omega * Z

In [12]:
np.round(svt_algo(Z, omega), 2)

array([[-0.  ,  1.  ,  2.01,  2.73,  4.03,  5.04,  5.96,  7.06,  8.07,
         9.08],
       [ 9.99, 11.  , 12.01, 12.77, 14.03, 15.04, 15.97, 17.05, 18.06,
        19.07],
       [20.06, 21.05, 22.04, 22.82, 24.01, 25.  , 25.94, 26.97, 27.96,
        28.95],
       [30.13, 31.11, 32.08, 32.9 , 34.02, 35.  , 35.93, 36.94, 37.92,
        38.89],
       [40.05, 41.04, 42.03, 43.  , 44.01, 45.  , 40.4 , 46.98, 47.97,
        48.96],
       [49.98, 50.99, 51.99, 52.92, 54.01, 55.01, 56.  , 57.02, 58.03,
        59.03],
       [59.98, 60.99, 61.99, 62.96, 64.  , 65.01, 66.01, 67.02, 68.02,
        69.03],
       [69.98, 70.98, 71.99, 73.  , 74.  , 75.  , 76.01, 77.01, 78.01,
        79.02],
       [79.98, 80.98, 81.98, 83.04, 83.99, 84.99, 86.02, 87.  , 88.01,
        89.01],
       [89.97, 90.98, 91.98, 93.07, 93.99, 94.99, 96.03, 96.99, 98.  ,
        99.  ]])

In [13]:
np.round(svt_algo(Z, omega), 1) - np.arange(100).reshape(10,10)

array([[-0. ,  0. ,  0. , -0.3,  0. ,  0. ,  0. ,  0.1,  0.1,  0.1],
       [ 0. ,  0. ,  0. , -0.2,  0. ,  0. ,  0. ,  0.1,  0.1,  0.1],
       [ 0.1,  0. ,  0. , -0.2,  0. ,  0. , -0.1,  0. ,  0. , -0.1],
       [ 0.1,  0.1,  0.1, -0.1,  0. ,  0. , -0.1, -0.1, -0.1, -0.1],
       [ 0.1,  0. ,  0. ,  0. ,  0. ,  0. , -5.6,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

Видно, что в этот раз результат получился гораздо лучше, мы c гораздо большей точностью угадали затертые числа.

In [14]:
Z = np.arange(100).reshape(10,10)
Z = Z % 20
omega = np.ones(100).reshape(10,10)
omega[2, 3] = 0 #маска, явно указывает, какие числа мы как будто бы стерли
omega[3, 3] = 0
omega[4, 3] = 0
omega[3, 5] = 0
omega[4, 5] = 0
print(Z)
print(projection(omega, Z))

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  0.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12.  0. 14.  0. 16. 17. 18. 19.]
 [ 0.  1.  2.  0.  4.  0.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]]


In [15]:
np.round(svt_algo(Z, omega), 1) - Z

array([[-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

Здесь мы смогли целиком восстановить все потерянные значения 

Рассмотрим какую-то дурацкую матрицу ранга два и посмотрим как на ней мы восстановимся.

In [40]:
Z = np.array([[1, -2, 1, 3, 4, 2, 1],
              [-1, 3, 0, 2, 0, -3, 1],
              [1, -1, 2, 8, 8, 1, 3],
              [2, -5, 1, 1, 4, 5, 0],
              [0, 1, 1, 5, 4, -1, 2],
              [4, -7, 5, 17, 20, 7, 6],
              [4, -9, 3, 7, 12, 9, 2]])

In [46]:
omega = np.ones(49).reshape(7, 7)
omega[2, 3] = 0 #маска, явно указывает, какие числа мы как будто бы стерли
omega[3, 3] = 0
omega[4, 3] = 0

In [48]:
np.round(svt_algo(Z, omega), 1) - Z

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -0.],
       [-0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [49]:
anti_projection(omega, np.round(svt_algo(Z, omega), 2)) +\
projection(omega, Z) - Z

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -0.04,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

Здесь мы смогли почти в точности угадать все числа, которых не было в матрице, что достаточно неплохо. 

In [16]:
Z = np.ones(30).reshape(3,10)
# Z = Z % 20
omega = np.ones(30).reshape(3,10)
omega[1] = np.zeros(10)
print(Z)
print(projection(omega, Z))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [17]:
np.round(svt_algo(Z, omega), 1) 

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

Понятно, что при добавлении абсолютно пустой строки ничего и не должно было произойти &mdash; пустое дополнение и есть решение нашей задачи.

### Вероятностный алгоритм уменьшения суммы квадратов

Рассмотрим теперь Alternating Least Squares алгоритм, основывающийся на статьях [Yehuda Koren, Robert Bell and Chris Volinsky](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5197422) и [Ruslan Salakhutdinov, Andriy Mnih](https://www.cs.toronto.edu/~amnih/papers/bpmf.pdf)

In [25]:
def calculate_iter_answer(U, V, beta, gamma):
    n = V.shape[0]
    m = U.shape[0]
    return np.dot(U, V.T) + \
           np.outer(beta, np.ones(n)) + \
           np.outer(np.ones(m), gamma)

def calculate_U(Z, CU, U, V, beta, gamma):
    n = V.shape[0]
    m = U.shape[0]
    k = V.shape[1]
    Z_tmp = Z - np.outer(np.ones(m), gamma)
    V_tmp = np.c_[np.ones(n), V]
    for i in range(m):
        U_tmp = np.linalg.solve(np.linalg.multi_dot([V_tmp.T,
                                                       CU[i],
                                                       V_tmp]) +
                                  mu * np.eye(k + 1),
                                  np.linalg.multi_dot([V_tmp.T,
                                                       CU[i],
                                                       Z_tmp[i,:]]))
        beta[i] = U_tmp[0]
        U[i] = U_tmp[1:]
    return U
    
def calculate_V(Z, CV, U, V, beta, gamma):
    n = V.shape[0]
    m = U.shape[0]
    k = V.shape[1]
    Z_tmp = Z - np.outer(beta, np.ones(n))
    U_tmp = np.c_[np.ones(m), U]
    for j in range(n):
        V_tmp = np.linalg.solve(np.linalg.multi_dot([U_tmp.T,
                                                       CV[j],
                                                       U_tmp]) +
                                  mu * np.eye(k + 1),
                                  np.linalg.multi_dot([U_tmp.T,
                                                       CV[j],
                                                       Z_tmp[:,j]]))
        gamma[j] = V_tmp[0]
        V[j] = V_tmp[1:]
    return V

    
def probabistic_algo(Z, mask, k, mu, eps=1e-3, max_iterations=100):
    m, n = Z.shape
    rdm_f = sps.norm.rvs 

    U = rdm_f(size = (m, k))
    V = rdm_f(size = (n, k))
    beta = rdm_f(size = m)
    gamma = rdm_f(size = n)

    CU = [np.diag(row) for row in mask]
    CV = [np.diag(col) for col in mask.T]

    Z_old = calculate_iter_answer(U, V, beta, gamma)
    
    for _ in range(max_iterations):
        U = calculate_U(Z, CU, U, V, beta, gamma)
        V = calculate_V(Z, CV, U, V, beta, gamma)
        X = calculate_iter_answer(U, V, beta, gamma)

        if np.linalg.norm(X - Z_old) / m / n < eps:
            break
        Z_old = X

    return Z_old


In [26]:
Z = np.arange(100).reshape(10,10)
omega = np.ones(100).reshape(10,10)
omega[2, 3] = 0
omega[3, 3] = 0
omega[4, 3] = 0
omega[3, 5] = 0
omega[4, 5] = 0

k=1
mu=0.1
anti_projection(omega, np.round(probabistic_algo(Z, omega, k, mu), 2)) +\
projection(omega, Z)

array([[ 0.  ,  1.  ,  2.  ,  3.  ,  4.  ,  5.  ,  6.  ,  7.  ,  8.  ,
         9.  ],
       [10.  , 11.  , 12.  , 13.  , 14.  , 15.  , 16.  , 17.  , 18.  ,
        19.  ],
       [20.  , 21.  , 22.  , 23.02, 24.  , 25.  , 26.  , 27.  , 28.  ,
        29.  ],
       [30.  , 31.  , 32.  , 32.96, 34.  , 34.92, 36.  , 37.  , 38.  ,
        39.  ],
       [40.  , 41.  , 42.  , 42.94, 44.  , 44.9 , 46.  , 47.  , 48.  ,
        49.  ],
       [50.  , 51.  , 52.  , 53.  , 54.  , 55.  , 56.  , 57.  , 58.  ,
        59.  ],
       [60.  , 61.  , 62.  , 63.  , 64.  , 65.  , 66.  , 67.  , 68.  ,
        69.  ],
       [70.  , 71.  , 72.  , 73.  , 74.  , 75.  , 76.  , 77.  , 78.  ,
        79.  ],
       [80.  , 81.  , 82.  , 83.  , 84.  , 85.  , 86.  , 87.  , 88.  ,
        89.  ],
       [90.  , 91.  , 92.  , 93.  , 94.  , 95.  , 96.  , 97.  , 98.  ,
        99.  ]])

Здесь мы смогли предсказать все числа с еще большей точностью, чем в предыдущих двух методах.

In [30]:
Z = np.arange(100).reshape(10,10)
Z = Z % 20
omega = np.ones(100).reshape(10,10)
omega[2, 3] = 0 #маска, явно указывает, какие числа мы как будто бы стерли
omega[3, 3] = 0
omega[4, 3] = 0
omega[3, 5] = 0
omega[4, 5] = 0
print(Z)
print(projection(omega, Z))

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]]
[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  0.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12.  0. 14.  0. 16. 17. 18. 19.]
 [ 0.  1.  2.  0.  4.  0.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14. 15. 16. 17. 18. 19.]]


In [32]:
np.round(probabistic_algo(Z, omega, k, mu), 1) - Z

array([[ 0.1,  0.1,  0.1,  0.1,  0. ,  0. ,  0. , -0.1, -0.1, -0.1],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.1,  0.1,  0.1,  0.1,  0. ,  0. ,  0. , -0.1, -0.1, -0.1],
       [-0.1, -0.1,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.1,  0.1,  0.1,  0.1,  0. ,  0. ,  0. , -0.1, -0.1, -0.1],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.1,  0.1,  0.1,  0.1,  0. ,  0. ,  0. , -0.1, -0.1, -0.1],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0.1,  0.1,  0.1,  0.1,  0. ,  0. ,  0. , -0.1, -0.1, -0.1],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

Рассмотрим теперь произвольную матрицу ранга 2, посмотрим как алгоритм восстановит её

In [33]:
Z = np.array([[1, -2, 1, 3, 4, 2, 1],
              [-1, 3, 0, 2, 0, -3, 1],
              [1, -1, 2, 8, 8, 1, 3],
              [2, -5, 1, 1, 4, 5, 0],
              [0, 1, 1, 5, 4, -1, 2],
              [4, -7, 5, 17, 20, 7, 6],
              [4, -9, 3, 7, 12, 9, 2]])

In [36]:
omega = np.ones(49).reshape(7, 7)
omega[2, 3] = 0 #маска, явно указывает, какие числа мы как будто бы стерли
omega[3, 3] = 0
omega[4, 3] = 0

In [37]:
np.round(probabistic_algo(Z, omega, k, mu), 1) - Z

array([[-0.6,  0.3, -0.2,  0.9,  0.5, -1.2,  0.2],
       [ 0.6, -0.4,  0.2, -1.3, -0.3,  1.1, -0.1],
       [ 0.7, -0.4,  0.1, -2.3, -1.4,  1.5, -0.6],
       [-1.4,  0.8, -0.2,  4.3,  2.7, -3. ,  1.1],
       [ 0.5, -0.3,  0. , -2. , -1.1,  1.2, -0.5],
       [ 1.7, -1. ,  0.5, -2.6, -1.5,  3.4, -0.6],
       [-1.5,  0.9, -0.5,  2.8,  1. , -2.9,  0.4]])

In [39]:
anti_projection(omega, np.round(probabistic_algo(Z, omega, k, mu), 2)) +\
projection(omega, Z) - Z

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -2.32,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  4.26,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , -1.97,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ]])

### Вывод

Мы просмотрели три решения задачи восстановления матриц. Soft-thresholding algorithm показал себя с хорошей стороны при решении задачи на какой-то более менее случайной матрице второго ранга, в которой мы стерли несколько чисел, тогда как Alternating Least Squares проявил себя лучше в задаче восстановления матрицы 10х10 с числами из большого диапазона. Стоит отметить, что при этом его вычисление представляется более сложной задачей с точки зрения как компьютерных ресурсов так и восприятия алгоритма.